In [2]:
import os
import subprocess
import shutil
from glob import glob
import nibabel as nib
import numpy as np
import torch
import os

In [3]:
affine_warps = sorted(glob(os.path.join('dataset3', 'Femina3', 'MRI', "*_affine.mat")))
print(len(affine_warps))
deformable_warps = sorted(glob(os.path.join('dataset3', 'Femina3', 'MRI', "*_warp.nii.gz")))
print(len(deformable_warps))
labels = sorted(glob(os.path.join('dataset3', 'Femina3', 'Labels2', "*_id.nii.gz")))
print(len(labels))
atlas = os.path.join('dataset3', 'Atlas', 'P56_Atlas_128_norm_id.nii.gz')

for i, label in enumerate(labels):
    # Apply .mat to Labels to get AFFINE LABELS          
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', '<Moving>'])
    command.extend(['-r', '<Fixed>'])
    command.extend(['-o', '<Output>'])
    command.extend(['-n', '<Interpolation>'])
    command.extend(['-t', '<AffineTransform>'])

    for j, cmd in enumerate(command):
        cmd = cmd.replace("<Fixed>", atlas)
        cmd = cmd.replace("<Moving>", labels[i])
        cmd = cmd.replace("<Output>", labels[i].replace("_id.nii", "_id_affine.nii"))            
        cmd = cmd.replace("<Interpolation>", "MultiLabel")
        cmd = cmd.replace("<AffineTransform>", affine_warps[i])  
        command[j] = cmd

    subprocess.call(command) 

    # Apply .mat and warp to Labels to get DEFORMABLE LABELS                          
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', '<Moving>'])
    command.extend(['-r', '<Fixed>'])
    command.extend(['-o', '<Output>'])
    command.extend(['-n', '<Interpolation>'])
    command.extend(['-t', '<DeformableTransform>'])
    command.extend(['-t', '<AffineTransform>'])

    for j, cmd in enumerate(command):
        cmd = cmd.replace("<Fixed>", atlas)
        cmd = cmd.replace("<Moving>", labels[i])
        cmd = cmd.replace("<Output>", labels[i].replace("_id.nii", "_id_affine_deformable.nii"))            
        cmd = cmd.replace("<Interpolation>", "MultiLabel")
        cmd = cmd.replace("<DeformableTransform>", deformable_warps[i])  
        cmd = cmd.replace("<AffineTransform>", affine_warps[i])  
        command[j] = cmd

    subprocess.call(command) 

135
135
135
